In [1]:
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
unknown_faces = np.load('unknown_faces.npy')
my_Faces = np.load('my_Faces.npy')

In [3]:
unknown_faces.shape

(400, 50, 50, 3)

In [4]:
unknown_faces = unknown_faces.reshape(400,-1)
my_faces = my_Faces.reshape(400,-1)

In [5]:
my_faces.shape

(400, 7500)

In [6]:
unknown_faces.shape

(400, 7500)

In [7]:
X = np.concatenate([my_faces, unknown_faces])

In [8]:
X.shape

(800, 7500)

In [10]:
labels = np.zeros(X.shape[0])

In [11]:
labels[400:] = 1.0

In [12]:
np.unique(labels, return_counts=True)

(array([0., 1.]), array([400, 400], dtype=int64))

In [13]:
# normalization
X = X / 255.0

In [14]:
from sklearn.model_selection import train_test_split

In [32]:
x_train, x_test, y_train, y_test = train_test_split(X,labels,test_size=0.25, random_state=100)

In [46]:
logistic = LogisticRegression(C=0.5, penalty='l1', solver='liblinear')
logistic.fit(x_train, y_train)

LogisticRegression(C=0.5, penalty='l1', solver='liblinear')

In [47]:
y_pred = logistic.predict(x_test)

In [48]:
accuracy_score(y_test, y_pred)

1.0

In [56]:
#capture = cv2.VideoCapture('video_1.mp4')
haar_data = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
capture = cv2.VideoCapture(0)
while True:
    flag, frame = capture.read()
    if flag:
        frame = cv2.resize(frame, None, fx=0.5, fy=0.5)
        faces = haar_data.detectMultiScale(frame, 1.3)
        for x,y,w,h in faces:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 3)
            
            face = frame[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            #face = face / 255.0
            pred = logistic.predict(face.reshape(1,-1))
            if pred == 0:
                name = "Ravi"
            else:
                name = "Unknown"
            cv2.putText(frame, name, (x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('result', frame)
        # 27 - ASCII of Escape
        if cv2.waitKey(1) == 27:
            break
    else:
        print("Camera/Video is missing")
cv2.destroyAllWindows()
capture.release()